# Create Label Xarray

In [1]:
start_date = '2022-01-01 00:00:00'
end_date = '2022-03-31 23:00:00'
freq='4H'
resolution = 0.05  # 0.1 degree

weight_dir=r'C:\Users\isxzl\OneDrive\Code\AutoGeo\weight'
site_path = r'C:\Datasets\Zhejiang20-23RS\site_withCoor.csv'  

grib_fea_path=r"C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\2022_Jan_2_Mar_feature.nc"
grib_predict_path=r"C:\Datasets\Zhejiang20-23RS\ERA5_featureRanking1\2022_Jan_2_Mar_predicted.nc"

In [2]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.dataset import create_xarray_within_boundary
import pandas as pd

# Example usage
boundary = (27, 118, 31.5, 122.5)  # min_latitude, min_longitude, max_latitude, max_longitude

# Generate a list of datetime objects from the start of 2022 to the end of March 2022 with hourly resolution
datetime_list = pd.date_range(start=start_date, end=end_date, freq=freq).tolist() 
ds = create_xarray_within_boundary(datetime_list, boundary, resolution) 

C:\Users\isxzl\anaconda3\envs\EO\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Model

In [3]:
import sys
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
from sample.prediction import load_model,predict_with_xarray
import xarray as xr

model,t1,t2,t3,features=load_model(weight_dir)
print(features)


Items have been loaded successfully.
['sdor', 'p53.162', 'anor', 'z', 'slor', 'swvl3', 'lmld', 'lsm', 'fsr', 'tp', 'srtm', 'sdfor', 'str', 'ssrd', 'ssr', 'tco3', 'chnk', 'vipie', 'sp', 'swvl4', 'src', 'viwve', 'vioze', 'cbh', 'lcc', 'time', 'longitude', 'latitude']


In [4]:
ds = xr.open_dataset(grib_fea_path)
ds = ds.drop_vars('spatial_ref')
x_predicted=predict_with_xarray(ds,features,[t1,t2,t3],model,'Negative_oxygen_ions')
x_predicted

Predicting:   0%|                                                                              | 0/540 [00:00<?, ?it/s]Exception in thread Thread-7 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\isxzl\anaconda3\envs\EO\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\isxzl\anaconda3\envs\EO\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\isxzl\anaconda3\envs\EO\lib\subprocess.py", line 1515, in _readerthread
    buffer.append(fh.read())
  File "C:\Users\isxzl\anaconda3\envs\EO\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 3: character maps to <undefined>
C:\Users\isxzl\anaconda3\envs\EO\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physi

<xarray.Dataset>
Dimensions:               (time: 540, latitude: 180, longitude: 180)
Coordinates:
  * time                  (time) datetime64[ns] 2022-01-01 ... 2022-03-31T20:...
  * latitude              (latitude) float64 27.0 27.02 27.05 ... 31.45 31.47
  * longitude             (longitude) float64 118.0 118.0 118.1 ... 122.5 122.5
    number                int32 0
    surface               float64 0.0
    step                  timedelta64[ns] 00:00:00
    entireAtmosphere      float64 0.0
    meanSea               float64 0.0
    depthBelowLandLayer   float64 7.0
Data variables: (12/128)
    Negative_oxygen_ions  (time, latitude, longitude) float64 ...
    deg0l                 (time, latitude, longitude) float64 ...
    zust                  (time, latitude, longitude) float64 ...
    ewss                  (time, latitude, longitude) float64 ...
    i10fg                 (time, latitude, longitude) float64 ...
    tplb                  (time, latitude, longitude) float64 ...
    ...                    ...
    lshf                  (time, latitude, longitude) float64 ...
    stl3                  (time, latitude, longitude) float64 ...
    stl4                  (time, latitude, longitude) float64 ...
    lblt                  (time, latitude, longitude) float64 ...
    srtm                  (time, latitude, longitude) float64 ...
    LC_Type4              (time, latitude, longitude) float64 ...

In [6]:
x_predicted.to_netcdf(, format='NETCDF4')